---
<center>

  # **Tarea 10: El átomo de Helio**

**Realizado Por:**

   Samuel Huertas Rojas

---
</center>

# 1.  
Use el método variacional con la base de estados 
$$
\Psi^{+} = \frac{1}{\sqrt{2}}\left( \lvert n_1\,00\,n_1' \,00 \rangle + \lvert n_1'\,00\,n_1\,00 \rangle \right)
$$
para obtener las correcciones radiales a la energía del estado fundamental. Hágalo para un máximo de 
$ n_1 $  y $ n_1' = 10 $.
(Resultado $ E_0 = -2.8752 $ u.a.).



Definición de la integrales directas y de intercambio: 

* Integral directa:

$$
\mathcal{I}_1 = \int_0^\infty dr_1 \, r_1^2 \, R_{n_1 0}(r_1) \, R_{n_2 0}(r_1) \int_0^\infty dr_2 \, r_2^2 \, R_{n_1' 0}(r_2) \, R_{n_2' 0}(r_2) \, \frac{1}{r_>}
$$

* Integral de intercambio:

$$
\mathcal{I}_2 = \int_0^\infty dr_1 \, r_1^2 \, R_{n_1 0}(r_1) \, R_{n_2' 0}(r_1) \int_0^\infty dr_2 \, r_2^2 \, R_{n_1' 0}(r_2) \, R_{n_2 0}(r_2) \, \frac{1}{r_>}
$$

In [1]:
import numpy as np
from scipy.integrate import dblquad
import sympy as sp
from functools import lru_cache
from sympy.physics.hydrogen import R_nl, E_nl

# ============================================================================
# CONFIGURACIÓN Y PARÁMETROS
# ============================================================================
ALPHA = 1 / 137.036
C = 1 / ALPHA

MU, E, HBAR = 1, 1, 1
Z = 2

FACTOR_CONVERSION = {"joul": 4.3598e-18, "cm": 219474.6313, "eV": 27.2114}

N1_MAX = 10
N2_MAX = 10

# Variables simbólicas
r1, r2 = sp.symbols("r1 r2", real=True, positive=True)


# ============================================================================
# FUNCIONES AUXILIARES
# ============================================================================
def factor_normalizacion(n1, n2, l1, l2, m1, m2):
    """
    Calcula el factor de normalización.

    Args:

    Returns:
        float: Factor de normalización
    """
    if n1 == n2 and l1 == l2 and m1 == m2:
        norma = 1 / 2
    else:
        norma = 1 / sp.sqrt(2)
    return norma


@lru_cache(maxsize=1000)
def calcular_integral_directa(n1, nl1, n2, nl2):
    """
    Calcula la integral directa I1 con caché para evitar recálculos.

    Args:
        n1, nl1, n2, nl2: Números cuánticos

    Returns:
        float: Valor de la integral
    """
    argumento = (
        (
            r1**2
            * r2**2
            * R_nl(nl2, 0, r2, Z)
            * R_nl(n2, 0, r1, Z)
            * R_nl(nl1, 0, r2, Z)
            * R_nl(n1, 0, r1, Z)
            / sp.Max(r1, r2)
        )
        * 2
        * factor_normalizacion(n1, nl1, 0, 0, 0, 0)
        * factor_normalizacion(n2, nl2, 0, 0, 0, 0)
    )

    fun = sp.lambdify((r1, r2), argumento)
    resultado, error = dblquad(fun, 0, 100, lambda x: 0, lambda x: 100)

    return resultado


@lru_cache(maxsize=1000)
def calcular_integral_intercambio(n1, nl1, n2, nl2):
    """
    Calcula la integral de intercambio I2 con caché.

    Args:
        n1, nl1, n2, nl2: Números cuánticos

    Returns:
        float: Valor de la integral
    """
    argumento = (
        (
            r1**2
            * r2**2
            * R_nl(nl2, 0, r1, Z)
            * R_nl(n1, 0, r1, Z)
            * R_nl(nl1, 0, r2, Z)
            * R_nl(n2, 0, r2, Z)
            / sp.Max(r1, r2)
        )
        * 2
        * factor_normalizacion(n1, nl1, 0, 0, 0, 0)
        * factor_normalizacion(n2, nl2, 0, 0, 0, 0)
    )

    fun = sp.lambdify((r1, r2), argumento)
    resultado, error = dblquad(fun, 0, 100, lambda x: 0, lambda x: 100)

    return resultado


# ============================================================================
# CONSTRUCCIÓN DE LA BASE DE ESTADOS
# ============================================================================
def construir_base_estados(n1_max, n2_max):
    """
    Construye la lista de estados base.

    Args:
        n1_max, n2_max: Valores máximos de n1 y n2

    Returns:
        tuple: (lista_estados, num_estados)
    """
    lista_estados = []
    contador = 0

    lista_n1 = []
    lista_n2 = []

    for n1 in range(1, n1_max + 1):
        for n2 in range(n1, n2_max + 1):
            lista_n1.append(n1)
            lista_n2.append(n2)
            lista_estados.append((n1, n2))
            contador += 1

    num_estados = len(lista_estados)
    # print(lista_estados)
    print(f"Número de estados para realizar el cálculo: {num_estados}")

    return lista_estados, lista_n1, lista_n2, num_estados


# ============================================================================
# CONSTRUCCIÓN DEL HAMILTONIANO
# ============================================================================
def construir_hamiltoniano_orden_cero(lista_estados, num_estados):
    """
    Construye la matriz del Hamiltoniano de orden cero (parte diagonal).

    Args:
        estados: Lista de tuplas (n1, n2)
        num_estados: Número total de estados

    Returns:
        np.ndarray: Matriz del Hamiltoniano H0
    """
    # Crear la matriz del hamiltoniano
    H0 = np.zeros((num_estados, num_estados))

    # Construccion de la matriz del hamiltoniano de orden cero
    for i in range(0, num_estados):
        n1, n2 = lista_estados[i]
        H0[i][i] = -0.5 * Z**2 * (1 / n1**2 + 1 / n2**2)

    autovalores_H0, autovectores_H0 = np.linalg.eig(H0)

    return H0

def agregar_perturbacion(H, estados):

    """
    Agrega las integrales directas y de intercambio al Hamiltoniano.

    Args:
        H: Matriz del Hamiltoniano (se modifica in-place)
        estados: Lista de estados
    """
    num_estados = len(estados)

    print("\nCalculando elementos de matriz...")
    # total_elementos = num_estados * (num_estados + 1) // 2
    # contador_elementos = 0

    for i in range(num_estados):
        n1_i, n2_i = estados[i]

        for j in range(i, num_estados):
            n1_j, n2_j = estados[j]

            # Calcular integrales
            I1 = calcular_integral_directa(n1_i, n2_i, n1_j, n2_j)
            I2 = calcular_integral_intercambio(n1_i, n2_i, n1_j, n2_j)

            # Actualizar matriz (aprovechando simetría, ya que es hermitica)
            H[i, j] = H[i, j] + I1 + I2
            H[j, i] = H[i, j]

            # Progreso
            # contador_elementos += 1
            # if contador_elementos % 10 == 0:
            #     porcentaje = 100 * contador_elementos / total_elementos
            #     print(
            #         f"Progreso: {porcentaje:.1f}% ({contador_elementos}/{total_elementos})"
            #     )

            # print(f"<{n1_i},{n2_i}|{n1_j},{n2_j}> = {H[i, j]:.6f}")


# ============================================================================
# FUNCIÓN PRINCIPAL
# ============================================================================
def main():
    """
    Función principal que ejecuta el cálculo completo.
    """
    # Construir base de estados
    estados, lista_n1, lista_n2, num_estados = construir_base_estados(N1_MAX, N2_MAX)

    # Construir Hamiltoniano de orden cero
    H = construir_hamiltoniano_orden_cero(estados, num_estados)

    # Diagonalizar H0
    print("\nDiagonalizando Hamiltoniano de orden cero...")
    autovalores_H0, autovectores_H0 = np.linalg.eigh(H)
    print(f"\nAutovalores H0 (primeros 5):\n{autovalores_H0[:5]}")

    # Agregar perturbación
    agregar_perturbacion(H, estados)

    # Diagonalizar Hamiltoniano completo
    print("\nDiagonalizando Hamiltoniano completo...")
    autovalores, autovectores = np.linalg.eigh(H)

    print(f"\nAutovalores finales (primeros 10):\n{autovalores[:10]}")
    print(f"\nEnergía del estado fundamental: {autovalores[0]:.6f}")


In [2]:
main()

Número de estados para realizar el cálculo: 55

Diagonalizando Hamiltoniano de orden cero...

Autovalores H0 (primeros 5):
[-4.         -2.5        -2.22222222 -2.125      -2.08      ]

Calculando elementos de matriz...


c:\Users\samhu\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\integrate\_quadpack_py.py:1260: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,
c:\Users\samhu\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\integrate\_quadpack_py.py:1260: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,
c:\Users\samhu\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\integrate\_quadpack_py


Diagonalizando Hamiltoniano completo...

Autovalores finales (primeros 10):
[-2.84542943 -2.13761711 -2.05868712 -2.03250296 -2.02062798 -2.01425577
 -2.01078041 -2.0064424  -1.99023731 -1.6980686 ]

Energía del estado fundamental: -2.845429


# 2.
Use ahora la base de estados singlete excitados discretos, es decir,
$
        \Psi^{+} = \frac{1}{\sqrt{2}}
        \left(
            \lvert 100\,n l m \rangle + \lvert n l m\,100 \rangle 
        \right)
$
hasta $n = 3 $ para obtener la energía del estado fundamental del Helio.
    (Resultado $ E_0 = 2.83865 $ u.a.). A partir de la matriz construida, muestre cuál es la energía perturbada a primer orden de los estados excitados.


Definición de la integrales directas y de intercambio: 

* Integral directa:

$$
\mathcal{I}_1 = \int_0^\infty dr_1 \, r_1^2 \, R_{1 0}(r_1) \, R_{1 0}(r_1) \int_0^\infty dr_2 \, r_2^2 \, R_{n' l'}(r_2) \, R_{n l}(r_2) \, \frac{1}{r_>} \delta_{l' l} \delta_{m' m}
$$

* Integral de intercambio:

$$
\mathcal{I}_2 = \dfrac{1}{2l +1} \int_0^\infty dr_1 \, r_1^2 \, R_{1 0}(r_1) \, R_{n l}(r_1) \int_0^\infty dr_2 \, r_2^2 \, R_{n' l'}(r_2) \, R_{1 0}(r_2) \, \frac{r_<^l}{r_>^{l+1}} \delta_{l' l} \delta_{m' m}
$$

In [ ]:
@lru_cache(maxsize=1000)
def calcular_integral_directa2(n, l, m, np, lp, mp):  # noqa: E741
    """
    Calcula la integral directa I1 con caché para evitar recálculos.

    Args:
        n1, nl1, n2, nl2: Números cuánticos

    Returns:
        float: Valor de la integral
    """
    argumento = (
        (
            r1**2
            * r2**2
            * R_nl(1, 0, r1, Z)
            * R_nl(1, 0, r1, Z)
            * R_nl(np, lp, r2, Z)
            * R_nl(n, l, r2, Z)
            / sp.Max(r1, r2)
        )
        * sp.KroneckerDelta(lp, l)
        * sp.KroneckerDelta(mp, m)
        * factor_normalizacion(n, 1, l, 0, m, 0)
        * factor_normalizacion(np, 1, lp, 0, mp, 0)
    )

    fun = sp.lambdify((r1, r2), argumento)
    resultado, error = dblquad(fun, 0, 100, lambda x: 0, lambda x: 100)

    return resultado


@lru_cache(maxsize=1000)
def calcular_integral_intercambio2(n, l, m, np, lp, mp):  # noqa: E741
    """
    Calcula la integral de intercambio I2 con caché.

    Args:
        n1, nl1, n2, nl2: Números cuánticos

    Returns:
        float: Valor de la integral
    """
    argumento = (
        (
            r1**2
            * r2**2
            * R_nl(1, 0, r1, Z)
            * R_nl(n, l, r1, Z)
            * R_nl(np, lp, r2, Z)
            * R_nl(1, 0, r2, Z)
            * sp.Min(r1, r2) ** l
            / (sp.Max(r1, r2) ** (l + 1))
        )
        * sp.KroneckerDelta(lp, l)
        * sp.KroneckerDelta(mp, m)
        * factor_normalizacion(n, 1, l, 0, m, 0)
        * factor_normalizacion(np, 1, lp, 0, mp, 0)
    )

    fun = sp.lambdify((r1, r2), argumento)
    resultado, error = dblquad(fun, 0, 100, lambda x: 0, lambda x: 100)

    return resultado


def construir_hamiltoniano_orden_cero2(lista_estados, num_estados):
    """
    Construye la matriz del Hamiltoniano de orden cero (parte diagonal).

    Args:
        estados: Lista de tuplas (n1, n2)
        num_estados: Número total de estados

    Returns:
        np.ndarray: Matriz del Hamiltoniano H0
    """
    # Crear la matriz del hamiltoniano
    H0 = np.zeros((num_estados, num_estados))

    # Construccion de la matriz del hamiltoniano de orden cero
    for i in range(0, num_estados):
        n, l, m = lista_estados[i]  # noqa: E741
        H0[i][i] = E_nl(1, Z) + E_nl(n, Z)

    return H0


# ============================================================================
# CONSTRUCCIÓN DE LA BASE DE ESTADOS
# ============================================================================
def construir_base_estados2(n_max):
    """
    Construye la lista de estados base.

    Args:
        n_max: Valores máximos de n1 y n2

    Returns:
        tuple: (lista_estados, num_estados)
    """
    lista_estados = []

    for n in range(1, n_max + 1):
        for l in range(n):  # noqa: E741
            for m in range(-l, l + 1):
                lista_estados.append((n, l, m))

    num_estados = len(lista_estados)
    # print(lista_estados)
    print(f"Número de estados para realizar el cálculo: {num_estados}")

    return lista_estados, num_estados


def agregar_perturbacion_singleton_exitados(H, estados):
    """
    Agrega las integrales directas y de intercambio al Hamiltoniano.

    Args:
        H: Matriz del Hamiltoniano (se modifica in-place)
        estados: Lista de estados
    """
    num_estados = len(estados)

    print("\nCalculando elementos de matriz...")
    # total_elementos = num_estados * (num_estados + 1) // 2
    # contador_elementos = 0

    for i in range(num_estados):
        n, l, m = estados[i]  # noqa: E741

        for j in range(i, num_estados):
            np, lp, mp = estados[j]

            # Calcular integrales
            I1 = calcular_integral_directa2(n, l, m, np, lp, mp)
            I2 = calcular_integral_intercambio2(n, l, m, np, lp, mp)

            #print(f"I1 + I2={I1 + I2}")

            # Actualizar matriz (aprovechando simetría, ya que es hermitica)
            H[i, j] = H[i, j] +  2 * (I1 + I2) 
            H[j, i] = H[i, j]

            # Progreso
            # contador_elementos += 1
            # if contador_elementos % 10 == 0:
            #     porcentaje = 100 * contador_elementos / total_elementos
            #     print(
            #         f"Progreso: {porcentaje:.1f}% ({contador_elementos}/{total_elementos})"
            #     )

            #print(f"<{n},{l},{m}|{np},{lp},{mp}> = {H[i, j]:.6f}")


# ============================================================================
# FUNCIÓN PRINCIPAL
# ============================================================================
def main2():
    """
    Función principal que ejecuta el cálculo completo.
    """
    # Construir base de estados
    estados, num_estados = construir_base_estados2(3)

    # Construir Hamiltoniano de orden cero
    H = construir_hamiltoniano_orden_cero2(estados, num_estados)

    # Diagonalizar H0
    print("\nDiagonalizando Hamiltoniano de orden cero...")
    autovalores_H0, autovectores_H0 = np.linalg.eigh(H)
    print(f"\nAutovalores H0 (primeros 5):\n{autovalores_H0[:5]}")

    # Agregar perturbación
    agregar_perturbacion_singleton_exitados(H, estados)

    # Diagonalizar Hamiltoniano completo
    print("\nDiagonalizando Hamiltoniano completo...")
    autovalores, autovectores = np.linalg.eigh(H)

    print(f"\nTamaño de la matriz H:\n{H.shape}")
    print(f"\nAutovalores finales:\n{autovalores}")
    print(f"\nEnergía del estado fundamental: {autovalores[0]:.5f}")


In [4]:
main2()

Número de estados para realizar el cálculo: 14

Diagonalizando Hamiltoniano de orden cero...

Autovalores H0 (primeros 5):
[-4.  -2.5 -2.5 -2.5 -2.5]

Calculando elementos de matriz...

Diagonalizando Hamiltoniano completo...

Autovalores finales:
[-2.83864846 -2.13619337 -2.11513518 -2.11513518 -2.11513518 -1.9977044
 -1.9977044  -1.9977044  -1.9977044  -1.9977044  -1.82322132 -1.77422439
 -1.77422439 -1.77422439]

Energía del estado fundamental: -2.83865


# 3.
Haga lo mismo que en el punto anterior para obtener la energía perturbada a primer orden de los estados triplete.


In [27]:
# ============================================================================
# FUNCIÓN PARA CONSTRUIR LOS ESTADOS BASE
# ============================================================================   
def construir_base_estados3(n_max):
    """
    Construye la lista de estados base.

    Args:
        n_max: Valores máximos de n1 y n2

    Returns:
        tuple: (lista_estados, num_estados)
    """
    lista_estados = []

    for n in range(1, n_max + 1):
        for l in range(n):  # noqa: E741
            for m in range(-l, l + 1):
                lista_estados.append((n, l, m))

    num_estados = len(lista_estados)
    # print(lista_estados)
    print(f"Número de estados para realizar el cálculo: {num_estados}")

    return lista_estados, num_estados

# ============================================================================
# FUNCIÓN PARA CONSTRUIR EL HAMILTONIANO DE ORDEN CERO
# ============================================================================   
def construir_hamiltoniano_orden_cero3(lista_estados):
    """
    Construye la matriz del Hamiltoniano de orden cero (parte diagonal).

    Args:
        estados: Lista de tuplas (n1, n2)
        num_estados: Número total de estados

    Returns:
        np.ndarray: Matriz del Hamiltoniano H0
    """
    num_estados = len(lista_estados)

    # Crear la matriz del hamiltoniano
    H0 = np.zeros((num_estados, num_estados))

    # Construccion de la matriz del hamiltoniano de orden cero
    for i in range(0, num_estados):
        n, l, m = lista_estados[i]  # noqa: E741
        H0[i][i] = E_nl(1, Z) + E_nl(n, Z)

    return H0

# ============================================================================
# FUNCIÓN PARA AGREGAR LOS ESTADOS TRIPLETE EXCITADOS
# ============================================================================   
def agregar_perturbacion_triplete_exitados(H, estados):
    """
    Agrega las integrales directas y de intercambio al Hamiltoniano.

    Args:
        H: Matriz del Hamiltoniano (se modifica in-place)
        estados: Lista de estados
    """
    num_estados = len(estados)

    print("\nCalculando elementos de matriz...")
    total_elementos = num_estados * (num_estados + 1) // 2
    contador_elementos = 0

    for i in range(num_estados):
        n, l, m = estados[i]  # noqa: E741
        for j in range(i, num_estados):
            np, lp, mp = estados[j]
            # Calcular integrales
            I1 = calcular_integral_directa2(n, l, m, np, lp, mp)
            I2 = calcular_integral_intercambio2(n, l, m, np, lp, mp)
            #print(f"I1 - I2={I1 - I2}")

            # Actualizar matriz (aprovechando simetría, ya que es hermitica)
            H[i, j] = H[i, j] + 2 * (I1 - I2)
            H[j, i] = H[i, j]

            # Progreso
            contador_elementos += 1
            if contador_elementos % 10 == 0:
                porcentaje = 100 * contador_elementos / total_elementos
                print(
                    f"Progreso: {porcentaje:.1f}% ({contador_elementos}/{total_elementos})"
                    )

            #print(f"<100,{n}{l}{m}|100,{np}{lp}{mp}> = {H[i, j]:.6f}")


# ============================================================================
# FUNCIÓN PARA FILTRAR LOS ESTADOS
# ============================================================================        
def filtrar_estados_y_matriz(estados, H, condicion):
    """
    Filtra elementos de la matriz según una condición entre pares de estados.
    
    Args:
        estados: Lista de tuplas (n, l, m)
        H: Matriz Hamiltoniana original
        condicion: Función que toma dos estados y retorna True para mantener el elemento
        
    Returns:
        estados_filtrados, H_filtrada, indices_eliminados
    """
    num_estados = len(estados)
    
    # Crear matriz de máscaras (True = mantener, False = eliminar)
    mascara = np.ones((num_estados, num_estados), dtype=bool)
    
    # Evaluar condición para cada par de estados
    for i in range(num_estados):
        for j in range(num_estados):
            if condicion(estados[i], estados[j]):
                mascara[i, j] = False
    
    # Identificar filas/columnas a eliminar completamente
    # (aquellas donde todos los elementos son False)
    filas_mantener = np.any(mascara, axis=1)
    columnas_mantener = np.any(mascara, axis=0)
    
    # Combinar: mantener solo filas Y columnas que tienen al menos un True
    indices_mantener = np.where(filas_mantener & columnas_mantener)[0]
    
    # Filtrar estados
    estados_filtrados = [estados[i] for i in indices_mantener]
    
    # Filtrar matriz
    H_filtrada = H[np.ix_(indices_mantener, indices_mantener)]
    
    # Información de estados eliminados
    indices_eliminados = [i for i in range(num_estados) if i not in indices_mantener]
    
    print(f"\nEstados eliminados ({len(indices_eliminados)}):")
    for idx in indices_eliminados:
        print(f"  Índice {idx}: {estados[idx]}")
    
    return estados_filtrados, H_filtrada, indices_eliminados

# ============================================================================
# FILTRO DE ESTADOS
# ============================================================================   
def condicion_filtro(estado1, estado2):
    n1, l1, m1 = estado1
    n2, l2, m2 = estado2
    
    # Retorna True si los estados son diferentes
    return (n1 == n2 and l1 == l2 and m1 == m2)


# ============================================================================
# FUNCIÓN PRINCIPAL
# ============================================================================
def main3():
    """
    Función principal que ejecuta el cálculo completo.
    """
    # Construir base de estados
    estados, num_estados = construir_base_estados3(3)
    estados = estados[1:]
    #print(f"Estados utilizados: {estados}")

    # Construir Hamiltoniano de orden cero
    H1 = construir_hamiltoniano_orden_cero3(estados)

    # Diagonalizar H0
    print("\nDiagonalizando Hamiltoniano de orden cero...")
    autovalores_H0, autovectores_H0 = np.linalg.eigh(H1)
    print(f"\nAutovalores H0 (primeros 5):\n{autovalores_H0[:5]}")

    # Agregar perturbación
    agregar_perturbacion_triplete_exitados(H1, estados)

    # Realizar el filtrado de los estados
    print("\n" + "="*70)
    print("OPCIÓN 2: Eliminar filas/columnas de estados idénticos")
    print("="*70)
    estados_2, H2, eliminados = filtrar_estados_y_matriz(
        estados, H1, condicion_filtro
    )
    print(f"Resultado: {H2.shape}, {len(estados_2)} estados")

    # Diagonalizar Hamiltoniano completo
    print("\nDiagonalizando Hamiltoniano completo...")
    autovalores, autovectores = np.linalg.eigh(H2)

    print(f"\nAutovalores finales:\n{autovalores}")
    print(f"\nEnergía del estado fundamental: {autovalores[0]:.6f}")

In [28]:
main3()

Número de estados para realizar el cálculo: 14

Diagonalizando Hamiltoniano de orden cero...

Autovalores H0 (primeros 5):
[-2.5        -2.5        -2.5        -2.5        -2.22222222]

Calculando elementos de matriz...
Progreso: 11.0% (10/91)
Progreso: 22.0% (20/91)
Progreso: 33.0% (30/91)
Progreso: 44.0% (40/91)
Progreso: 54.9% (50/91)
Progreso: 65.9% (60/91)
Progreso: 76.9% (70/91)
Progreso: 87.9% (80/91)
Progreso: 98.9% (90/91)

OPCIÓN 2: Eliminar filas/columnas de estados idénticos

Estados eliminados (0):
Resultado: (13, 13), 13 estados

Diagonalizando Hamiltoniano completo...

Autovalores finales:
[-2.16988063 -2.14970806 -2.14970806 -2.14970806 -2.00264825 -2.00264825
 -2.00264825 -2.00264825 -2.00264825 -1.99888068 -1.99888068 -1.99888068
 -1.9890453 ]

Energía del estado fundamental: -2.169881


# 4.
Use ahora la base 
$
        \frac{1}{\sqrt{2}} \left(
            \lvert n\,00\,n' \,00 \rangle 
            + 
            \lvert n'\,00\,n\,00 \rangle
        \right)
$
y encuentre las energías de estados triplete.

In [7]:
def construir_base_estados4(n1_max, n2_max):
    """
    Construye la lista de estados base.

    Args:
        n1_max, n2_max: Valores máximos de n1 y n2

    Returns:
        tuple: (lista_estados, num_estados)
    """
    lista_estados = []
    contador = 0

    lista_n1 = []
    lista_n2 = []

    for n1 in range(1, n1_max + 1):
        for n2 in range(n1, n2_max + 1):
            if n1 != n2:
                lista_n1.append(n1)
                lista_n2.append(n2)
                lista_estados.append((n1, n2))
                contador += 1

    num_estados = len(lista_estados)
    #print(lista_estados)
    print(f"Número de estados para realizar el cálculo: {num_estados}")

    return lista_estados, lista_n1, lista_n2, num_estados

def agregar_perturbacion_triplete(H, estados):

    """
    Agrega las integrales directas y de intercambio al Hamiltoniano.

    Args:
        H: Matriz del Hamiltoniano (se modifica in-place)
        estados: Lista de estados
    """
    num_estados = len(estados)

    print("\nCalculando elementos de matriz...")
    # total_elementos = num_estados * (num_estados + 1) // 2
    # contador_elementos = 0

    for i in range(num_estados):
        n1_i, n2_i = estados[i]

        for j in range(i, num_estados):
            n1_j, n2_j = estados[j]
            
            if n1_i != n2_i and n1_j != n2_j:
                # Calcular integrales
                I1 = calcular_integral_directa(n1_i, n2_i, n1_j, n2_j)
                I2 = calcular_integral_intercambio(n1_i, n2_i, n1_j, n2_j)

                # Actualizar matriz (aprovechando simetría, ya que es hermitica)
                H[i, j] = H[i, j] + I1 - I2
                H[j, i] = H[i, j]

                # Progreso
                # contador_elementos += 1
                # if contador_elementos % 10 == 0:
                #     porcentaje = 100 * contador_elementos / total_elementos
                #     print(
                #         f"Progreso: {porcentaje:.1f}% ({contador_elementos}/{total_elementos})"
                #     )

            #print(f"<{n1_i},{n2_i}|{n1_j},{n2_j}> = {H[i, j]:.6f}")


# ============================================================================
# FUNCIÓN PRINCIPAL
# ============================================================================
def main4():
    """
    Función principal que ejecuta el cálculo completo.
    """
    # Construir base de estados
    estados, lista_n1, lista_n2, num_estados = construir_base_estados4(4, 4)

    # Construir Hamiltoniano de orden cero
    H = construir_hamiltoniano_orden_cero(estados, num_estados)

    # Diagonalizar H0
    print("\nDiagonalizando Hamiltoniano de orden cero...")
    autovalores_H0, autovectores_H0 = np.linalg.eigh(H)
    print(f"\nAutovalores H0 (primeros 5):\n{autovalores_H0[:5]}")

    # Agregar perturbación
    agregar_perturbacion_triplete(H, estados)

    print(f"Tamaño de la matriz Hamiltoniano: {H.shape}")

    # Diagonalizar Hamiltoniano completo
    print("\nDiagonalizando Hamiltoniano completo...")
    autovalores, autovectores = np.linalg.eigh(H)

    print(f"\nAutovalores finales:\n{autovalores}")
    print(f"\nEnergía del estado fundamental: {autovalores[0]:.6f}")


In [8]:
main4()

Número de estados para realizar el cálculo: 6

Diagonalizando Hamiltoniano de orden cero...

Autovalores H0 (primeros 5):
[-2.5        -2.22222222 -2.125      -0.72222222 -0.625     ]

Calculando elementos de matriz...
Tamaño de la matriz Hamiltoniano: (6, 6)

Diagonalizando Hamiltoniano completo...

Autovalores finales:
[-2.17096146 -2.06733095 -1.93504854 -0.58439719 -0.51164409 -0.26494767]

Energía del estado fundamental: -2.170961
